## S3 Data Analysis Notebook

This notebook analyzes historical bid-ask data downloaded using `amp/im_v2/binance/data/extract/download_historical_bid_ask.py` and stored in an S3 bucket. It retrieves and processes the data, and outputs a summary of the data availability and missing dates.


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# Importing modules.
import logging
from datetime import datetime, timedelta
from typing import Any, Dict, List

import pandas as pd
from IPython.display import display

import helpers.haws as haws
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import helpers.hs3 as hs3

In [3]:
hdbg.init_logger(verbosity=logging.INFO)
log_level = logging.INFO

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO  > cmd='/venv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-61b8e7dc-09bd-4da8-b4cb-8539bfaf90c0.json'
INFO  # Git
  branch_name='CmampTask8422_Write_script_to_analyze_the_current_status_of_bid_ask_historical_data'
  hash='a904b1a73'
  # Last commits:
    * a904b1a73 sonaalKant checkpoint                                                        (    2 days ago) Sat Jun 8 04:47:35 2024  (HEAD -> CmampTask8422_Write_script_to_analyze_the_current_status_of_bid_ask_historical_data, origin/CmampTask8422_Write_script_to_analyze_the_current_status_of_bid_ask_historical_data)
    *   abfeab16e sonaalKant Merge branch 'master' of github.com:cryptokaizen/cmamp into CmampTask8422_Write_script_to_analyze_the_current_status_of_bid_ask_historical_data (    2 days ago) Sat Jun 8 04:00:06 2024           
    |\  
    * | a5ff5c10f sonaalKant checkpoint                                                        (    2 days ago) Sat Jun 8 03:59:48 2024    

## Configuration

#### Update the configuration section with the appropriate parameters to customize the analysis.

Configuration
config = {
    "stage": "preprod",
    "save_path_prefix": "binance/historical_bid_ask/",
}

## Helper Functions

#### The following functions help in listing and processing S3 objects.

In [5]:
# Extract the directory structure.
def extract_data_structure(objects):
    """
    Extract the directory structure and relevant data from the S3 objects.

    :param objects: S3 objects,
        e.g.
            ```
            [
                {
                    'Key': 'binance/historical_bid_ask/S_DEPTH/1000BONK_USDT/2023-05-27/data.tar.gz',
                    'LastModified': datetime.datetime(2024, 5, 30, 17, 12, 12, tzinfo=tzlocal()),
                    'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
                    'Size': 0,
                    'StorageClass': 'STANDARD'
                },
                {
                    'Key': 'binance/historical_bid_ask/S_DEPTH/1000BONK_USDT/2023-05-28/data.tar.gz',
                    'LastModified': datetime.datetime(2024, 5, 30, 17, 12, 12, tzinfo=tzlocal()),
                    'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
            ]
            ```
    """
    data_types = {}
    for obj in objects:
        key = obj["Key"]
        # Filter non-empty data.tar.gz files.
        if key.endswith("data.tar.gz") and obj["Size"] > 0:
            parts = key.split("/")
            date_part = parts[-2]
            date = datetime.strptime(date_part.split("_")[0], "%Y-%m-%d").date()
            symbol = parts[-3]
            data_type = parts[-4]
            #
            if data_type not in data_types:
                data_types[data_type] = {}
            #
            if symbol not in data_types[data_type]:
                data_types[data_type][symbol] = {"dates": [], "size": []}
            #
            data_types[data_type][symbol]["dates"].append(date)
            data_types[data_type][symbol]["size"].append(obj["Size"])

    return data_types

In [6]:
def group_consecutive_timestamps(
    timestamps: List[pd.Timestamp], threshold: timedelta = timedelta(days=1)
) -> List[List[pd.Timestamp]]:
    """
    Group consecutive timestamps for better readability. If the length of the
    list is 2, the first element in the list is the start timestamp and the
    second element is the end timestamp. If the length of the list is 1, then
    only a single timestamp is grouped.

    :param timestamps: List of pd.Timestamp objects to be grouped.
    :param threshold: Timedelta object representing the maximum allowed difference
                      between consecutive timestamps for them to be grouped together.
                      Defaults to 1 day.

    :return: List of lists of grouped timestamps. Each sublist contains either a single timestamp
             or two timestamps indicating the start and end of a consecutive period.

    Example:
        ```
        group_consecutive_timestamps([
            pd.Timestamp("2023-05-25"),
            pd.Timestamp("2023-05-27"),
            pd.Timestamp("2023-05-28"),
            pd.Timestamp("2023-05-30")
        ])
        ```
        Output:
        ```
        [
            [pd.Timestamp("2023-05-25")],
            [pd.Timestamp("2023-05-27"), pd.Timestamp("2023-05-28")],
            [pd.Timestamp("2023-05-30")]
        ]
        ```
    """
    # Sort the timestamps
    timestamps.sort()
    current_group = []
    groups = []
    for timestamp in timestamps:
        if not current_group:
            current_group.append(timestamp)
            continue
        # Check if the current timestamp is within the threshold of the last timestamp in the current group
        if (timestamp - current_group[-1]) <= threshold:
            current_group.append(timestamp)
        else:
            # Append the current group to the groups list
            if len(current_group) == 1:
                groups.append(current_group)
            else:
                groups.append([current_group[0], current_group[-1]])
            # Start a new group
            current_group = [timestamp]
    # Append the last group to the groups list
    if current_group:
        if len(current_group) == 1:
            groups.append(current_group)
        else:
            groups.append([current_group[0], current_group[-1]])
    return groups

In [7]:
def analyze_metadata(data_types: Dict[Dict, Any]) -> pd.DataFrame:
    """
    Analyze the metadata to find missing dates and generate a summary.
    """
    results = []
    for data_type, symbols in data_types.items():
        for symbol, data in symbols.items():
            min_date = min(data["dates"])
            max_date = max(data["dates"])
            all_dates = pd.date_range(start=min_date, end=max_date).date
            missing_dates = sorted(set(all_dates) - set(data["dates"]))
            missing_dates_groups = group_consecutive_timestamps(missing_dates)
            # We only want to verify downloaded data from `2023-05-27` as there was symantics change
            # by binance after this date.
            final_groups = []
            for group in missing_dates_groups:
                if pd.Timestamp(group[0]) <= pd.Timestamp("2023-05-27"):
                    continue
                final_groups.append(group)
            results.append(
                {
                    "data_type": data_type,
                    "symbol": symbol,
                    "min_date": min_date,
                    "max_date": max_date,
                    "missing_dates": final_groups,
                }
            )

    # Convert results to data frame.
    results_df = pd.DataFrame(results)
    return results_df

In [9]:
s3_client = haws.get_service_client(aws_profile="ck", service_name="s3")
bucket_name = hs3.get_s3_bucket_from_stage(stage=config["stage"])
prefix = config["save_path_prefix"]
# List objects in the S3 bucket under the specified prefix.
all_objects = haws.list_all_objects(s3_client, bucket_name, prefix)
_LOG.info(f"Length of all objects in s3 bucket is : {len(all_objects)}")
# Extract the directory structure.
data_types = extract_data_structure(all_objects)
# Analyze metadata.
results = analyze_metadata(data_types)
# Display the df.
display(results)

INFO  Length of all objects in s3 bucket is : 255315


,data_type,symbol,min_date,max_date,missing_dates
0,S_DEPTH,1000BONK_USDT,2023-11-22,2024-06-07,"[[2024-03-31], [2024-05-02, 2024-05-05]]"
1,S_DEPTH,1000FLOKI_USDT,2023-05-27,2024-06-07,"[[2023-11-09], [2023-11-13], [2024-05-03, 2024..."
2,S_DEPTH,1000LUNC_USDT,2023-05-27,2024-06-07,"[[2023-11-09], [2023-11-13]]"
3,S_DEPTH,1000PEPE_USDT,2023-05-27,2024-06-07,"[[2023-11-09], [2023-11-13], [2024-05-03, 2024..."
4,S_DEPTH,1000RATS_USDT,2023-12-15,2024-06-07,"[[2024-03-28], [2024-05-03, 2024-05-05]]"
...,...,...,...,...,...
521,T_DEPTH,ZEC_USDT,2020-07-05,2024-06-07,"[[2023-12-17], [2023-12-19], [2023-12-28, 2023..."
522,T_DEPTH,ZEN_USDT,2020-11-29,2024-06-07,"[[2023-11-26, 2023-11-27], [2023-11-29, 2023-1..."
523,T_DEPTH,ZETA_USDT,2023-11-29,2024-06-07,"[[2023-11-30, 2023-12-18], [2023-12-20, 2024-0..."
524,T_DEPTH,ZIL_USDT,2020-07-01,2024-06-07,"[[2023-12-28, 2023-12-31], [2024-01-17, 2024-0..."


In [10]:
T_DEPTH_df = results[results["data_type"] == "T_DEPTH"]
display(T_DEPTH_df)

,data_type,symbol,min_date,max_date,missing_dates
263,T_DEPTH,1000BONK_USDT,2023-11-22,2024-06-07,[]
264,T_DEPTH,1000FLOKI_USDT,2023-05-27,2024-06-07,[]
265,T_DEPTH,1000LUNC_USDT,2023-05-27,2024-06-07,[]
266,T_DEPTH,1000PEPE_USDT,2023-05-27,2024-06-07,[]
267,T_DEPTH,1000RATS_USDT,2023-11-26,2024-06-07,"[[2023-11-27, 2023-11-29], [2023-12-01, 2023-1..."
268,T_DEPTH,1000SATS_USDT,2023-11-18,2024-06-07,"[[2023-11-19, 2023-11-20], [2023-11-22, 2023-1..."
269,T_DEPTH,1000SHIB_USDT,2021-05-16,2024-06-07,[]
270,T_DEPTH,1000XEC_USDT,2023-05-27,2024-06-07,[]
271,T_DEPTH,1INCH_USDT,2020-12-27,2024-06-07,[]
272,T_DEPTH,AAVE_USDT,2020-10-18,2024-06-07,[]


In [11]:
S_DEPTH_df = results[results["data_type"] == "S_DEPTH"]
display(S_DEPTH_df)

,data_type,symbol,min_date,max_date,missing_dates
0,S_DEPTH,1000BONK_USDT,2023-11-22,2024-06-07,"[[2024-03-31], [2024-05-02, 2024-05-05]]"
1,S_DEPTH,1000FLOKI_USDT,2023-05-27,2024-06-07,"[[2023-11-09], [2023-11-13], [2024-05-03, 2024..."
2,S_DEPTH,1000LUNC_USDT,2023-05-27,2024-06-07,"[[2023-11-09], [2023-11-13]]"
3,S_DEPTH,1000PEPE_USDT,2023-05-27,2024-06-07,"[[2023-11-09], [2023-11-13], [2024-05-03, 2024..."
4,S_DEPTH,1000RATS_USDT,2023-12-15,2024-06-07,"[[2024-03-28], [2024-05-03, 2024-05-05]]"
5,S_DEPTH,1000SATS_USDT,2023-12-12,2024-06-07,"[[2024-03-25, 2024-03-27], [2024-05-03, 2024-0..."
6,S_DEPTH,1000SHIB_USDT,2023-05-27,2024-06-07,"[[2023-11-09], [2023-11-13], [2024-05-03, 2024..."
7,S_DEPTH,1000XEC_USDT,2023-05-27,2024-06-07,"[[2023-11-09], [2023-11-13], [2024-05-03, 2024..."
8,S_DEPTH,1INCH_USDT,2023-05-27,2024-06-07,"[[2023-11-09], [2023-11-13], [2024-05-03, 2024..."
9,S_DEPTH,AAVE_USDT,2023-05-27,2024-06-07,"[[2023-11-09], [2023-11-13], [2024-05-03, 2024..."
